In [ ]:
%matplotlib inline
#這是juoyter notebook的magic word˙

import matplotlib
import matplotlib.pyplot as plt
from IPython import display

# Covid 19醫療圖像檢測 資料預處理

In [ ]:
import os
#判斷是否在jupyter notebook上
def is_in_ipython():
    "Is the code running in the ipython environment (jupyter including)"
    program_name = os.path.basename(os.getenv('_', ''))

    if ('jupyter-notebook' in program_name or # jupyter-notebook
        'ipython'          in program_name or # ipython
        'jupyter' in program_name or  # jupyter
        'JPY_PARENT_PID'   in os.environ):    # ipython-notebook
        return True
    else:
        return False


#判斷是否在colab上
def is_in_colab():
    if not is_in_ipython(): return False
    try:
        from google import colab
        return True
    except: return False

#判斷是否在kaggke_kernal上
def is_in_kaggle_kernal():
    if 'kaggle' in os.environ['PYTHONPATH']:
        return True
    else:
        return False

if is_in_colab():
    from google.colab import drive
    drive.mount('/content/gdrive')

In [ ]:
os.environ['TRIDENT_BACKEND'] = 'pytorch'

if is_in_kaggle_kernal():
    os.environ['TRIDENT_HOME'] = './trident'
    
elif is_in_colab():
    os.environ['TRIDENT_HOME'] = '/content/gdrive/My Drive/trident'

#為確保安裝最新版 
!pip uninstall tridentx -y
!pip install tridentx --upgrade
!pip install pydicom --upgrade
import json
import copy
import numpy as np
#調用trident api
import trident as T
from trident import *
from trident.models import resnet,efficientnet
import random

* ### DICOM Tag
一份DICOM檔案中有許多的DICOM tag，用來詳細紀錄照片拍攝時的設備資訊、病人資訊等。Pydicom幫我們將DICOM tag轉換成python物件，讓我們更容易取得需要的資訊。下面程式利用pydicom讀取DICOM檔中的內容。pydicom將每一個tag條列如下面格式  
  
| DICOM tag(group,item) | DICOM tag 名稱 | DICOM tag 內容       |
|-----------------------|----------------|----------------------|
| (0010, 0020)          | Patient ID     | LO: 'LIDC-IDRI-0001' |

首先讀取一下看看全體圖像的數量

In [ ]:
dicom_images=glob.glob('../input/siim-covid19-detection/train/*/*/*.dcm')
print(len(dicom_images))

In [ ]:
import pydicom

example_case=random.choice(dicom_images)
print(example_case)
dicoms=pydicom.read_file(example_case)
print(dicoms)

我們可以利用pydicom.read_file(路徑)來讀取dicom檔，而取出dicom圖像也很簡單，直接讀取pixel_array屬性即可

In [ ]:
plt.rcParams["figure.figsize"] = (10,10)

#像素值並且儲存成numpy的格式
image = pydicom.read_file(example_case)
pixels=image.pixel_array
plt.imshow(pixels, cmap=plt.cm.gray)
plt.axis("off")
plt.show()


接下來要示範的是如何將模糊的圖變得清晰銳利

In [ ]:
from skimage.filters import unsharp_mask
from skimage import exposure

unsharp_image = unsharp_mask(pixels, radius=5, amount=2)
equalized_image = exposure.equalize_hist(unsharp_image)

plt.imshow(equalized_image, cmap=plt.cm.gray)
plt.axis("off")
plt.show()



In [ ]:
plt.hist(pydicom.read_file(example_case).pixel_array.flatten(), bins=80, color='c')
plt.xlabel("Hounsfield Units (HU)")
plt.ylabel("Frequency")
plt.show()

In [ ]:
#這裡的id是圖片的資料夾第一層
import pandas as pd
df_train_study=pd.read_csv('../input/siim-covid19-detection/train_study_level.csv', index_col='id')
df_train_study = df_train_study.idxmax(axis=1).to_frame(name='case')
df_train_study.index = df_train_study.index.str.replace('_study', '')
df_train_study

In [ ]:
df_train_image=pd.read_csv('../input/siim-covid19-detection/train_image_level.csv', index_col='id')
#df_train_image = df_train_image.merge(df_train_study, left_on='StudyInstanceUID', right_index=True)
df_train_image

In [ ]:
df_train_image = df_train_image.merge(df_train_study, left_on='StudyInstanceUID', right_index=True)
df_train_image

接下來是要解析Bounding Boxes

In [ ]:
boxes = []
for i, row in df_train_image.iterrows():
    value = row['boxes']
    if isinstance(value, float): # nan
        #print('x', end='')
        continue
    values = eval(value)
    #print('.', end='')
    for val in values:
        box = {
            'image': i,
            'study': row['StudyInstanceUID'],
            'case': row['case'],
            'xmin': val['x'],
            'ymin': val['y'],
            'xmax': val['x'] + val['width'],
            'ymax': val['y'] + val['height'],
            'width': val['width'],
            'height': val['height'],
        }
        boxes.append(box)
boxes = pd.DataFrame(boxes)
print(len(boxes))
boxes.head()
